In [2]:
!nvidia-smi

Thu Dec  8 21:08:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!git clone https://github.com/microsoft/GODEL

Cloning into 'GODEL'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 263 (delta 40), reused 36 (delta 30), pack-reused 203
Receiving objects: 100% (263/263), 51.03 MiB | 20.88 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [5]:
!pip install datasets
!pip install transformers
!pip install accelerate
!pip install fire
!pip install jsonlines
!pip install rouge_score
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
%cd GODEL/GODEL

/content/GODEL/GODEL


In [6]:
# Download Pre-trained Models
!wget https://bapengstorage.blob.core.windows.net/fileshare/godel_base.tar.gz
!tar -zxvf godel_base.tar.gz
!rm godel_base.tar.gz

--2022-12-08 21:08:56--  https://bapengstorage.blob.core.windows.net/fileshare/godel_base.tar.gz
Resolving bapengstorage.blob.core.windows.net (bapengstorage.blob.core.windows.net)... 20.150.87.36
Connecting to bapengstorage.blob.core.windows.net (bapengstorage.blob.core.windows.net)|20.150.87.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824827060 (787M) [application/gzip]
Saving to: ‘godel_base.tar.gz’

godel_base.tar.gz   100%[===================>] 786.62M  63.1MB/s    in 13s     

2022-12-08 21:09:09 (62.5 MB/s) - ‘godel_base.tar.gz’ saved [824827060/824827060]

GODEL-Base/
GODEL-Base/tokenizer.json
GODEL-Base/pytorch_model.bin
GODEL-Base/config.json
GODEL-Base/special_tokens_map.json
GODEL-Base/tokenizer_config.json
GODEL-Base/training_args.bin


In [7]:
# Generate SplitMemory Dataset
!gdown 1MkC5FmbRvk81EfpcHF9zASwYsUUKlQ7z
!tar -zxvf dialog_data.tar.gz
!rm dialog_data.tar.gz

Downloading...
From: https://drive.google.com/uc?id=1MkC5FmbRvk81EfpcHF9zASwYsUUKlQ7z
To: /content/GODEL/GODEL/dialog_data.tar.gz
100% 2.11G/2.11G [00:23<00:00, 91.4MB/s]
dialog_data/
dialog_data/train_data.jsonl
dialog_data/user_profiles.json
dialog_data/test_data.jsonl
dialog_data/valid_data.jsonl


In [2]:
import argparse
import copy
import fire
import logging
import math
import os
import random
import json
import jsonlines

import datasets
import nltk
import numpy as np
import torch
from datasets import load_dataset, load_metric
from torch.utils.data.dataloader import DataLoader

import transformers
from accelerate import Accelerator
from filelock import FileLock
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    SchedulerType,
    set_seed,
    get_scheduler,
    set_seed,
)
from transformers.file_utils import is_offline_mode
from tqdm.auto import tqdm
from utils.text_normalization import normalize_answer

In [3]:
# Preprocess PerChat Dataset

def process(filepath='./dialog_data'):
    user_profiles = json.load(open(f'{filepath}/user_profiles.json'))

    random.seed(2022)
    for folder in ['train', 'test', 'valid']:
      raw_data = []
      with jsonlines.open(f'{filepath}/{folder}_data.jsonl') as reader:
        for obj in reader:
          query = obj['src_text']
          for response in obj['responses']:
            data = {}
            data['Context'] = query
            data['Knowledge'] = ' EOS '.join(response['histories'])
            data['Response'] = response['response']
            raw_data.append(copy.deepcopy(data))

      with jsonlines.open(f'perchat_{folder}.jsonl', mode='w') as writer:
          for i in raw_data:
              if folder == 'train':
                  if random.random() < 0.01: # dataset too large, truncate
                      writer.write(i)
              else:
                  if random.random() < 0.1:
                      writer.write(i)

process()

In [4]:
!mkdir perchat

!mv perchat_train.jsonl perchat/
!mv perchat_valid.jsonl perchat/
!mv perchat_test.jsonl perchat/

mkdir: cannot create directory ‘perchat’: File exists


In [73]:
import datasets
import jsonlines

"""Corpus for PerChatPretrain"""

_DESCRIPTION = """\
PerChatPretrain
"""

_CITATION = """\
PerChatPretrain
"""

_WEBPAGE = ""

class PerChatPretrain(datasets.GeneratorBasedBuilder):
    """RedditPretrain"""

    def _info(self):
        return datasets.DatasetInfo(
            description=_DESCRIPTION,
            features=datasets.Features(
                {
                    "Context": datasets.Value("string"),
                    "Response": datasets.Value("string"),
                    "Knowledge": datasets.Value("string")
                }
            ),
            homepage=_WEBPAGE,
            citation=_CITATION,
        )

    def _split_generators(self, dl_manager):
        train_path = './perchat/perchat_train.jsonl'
        validation_path = './perchat/perchat_valid.jsonl'
        test_path = './perchat/perchat_test.jsonl'

        return [
            datasets.SplitGenerator(name=datasets.Split.TRAIN, gen_kwargs={
                                    "filepath": train_path}),
            datasets.SplitGenerator(name=datasets.Split.VALIDATION, gen_kwargs={
                                    "filepath": validation_path}),
            datasets.SplitGenerator(name=datasets.Split.TEST, gen_kwargs={
                                    "filepath": test_path}),
        ]

    def _generate_examples(self, filepath):
        key = 0
        with open(filepath, "r", encoding="utf-8") as reader:
            for item in jsonlines.Reader(reader):
                yield key, item
                key += 1



In [6]:
# Config
logging.getLogger().setLevel(logging.DEBUG)
logger = logging.getLogger(__name__)

if os.getenv('WANDB_API_KEY') is None:
    USE_WANDB = False
else:
    USE_WANDB = True
    wandb_key = os.getenv('WANDB_API_KEY')

MODEL_CONFIG_CLASSES = list(MODEL_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    if is_offline_mode():
        raise LookupError(
            "Offline mode: run this script without TRANSFORMERS_OFFLINE first to download nltk data files"
        )
    with FileLock(".lock") as lock:
        nltk.download("punkt", quiet=True)

In [33]:
def parse_args():
    parser = argparse.ArgumentParser(description="Finetune a transformers model on a text classification task")
    parser.add_argument(
        "--dataset_name",
        type=str,
        default=None,
        help="The name of the dataset to use (via the datasets library).",
    )
    parser.add_argument(
        "--dataset_config_name",
        type=str,
        default=None,
        help="The configuration name of the dataset to use (via the datasets library).",
    )
    parser.add_argument(
        "--train_file", type=str, default=None, help="A csv or a json file containing the training data."
    )
    parser.add_argument(
        "--validation_file", type=str, default=None, help="A csv or a json file containing the validation data."
    )
    parser.add_argument(
        "--max_source_length",
        type=int,
        default=1024,
        help="The maximum total input sequence length after "
        "tokenization.Sequences longer than this will be truncated, sequences shorter will be padded.",
    )
    parser.add_argument(
        "--source_prefix",
        type=str,
        default=None,
        help="A prefix to add before every source text " "(useful for T5 models).",
    )
    parser.add_argument(
        "--preprocessing_num_workers",
        type=int,
        default=None,
        help="The number of processes to use for the preprocessing.",
    )
    parser.add_argument(
        "--max_target_length",
        type=int,
        default=64,
        help="The maximum total sequence length for target text after "
        "tokenization. Sequences longer than this will be truncated, sequences shorter will be padded."
        "during ``evaluate`` and ``predict``.",
    )
    parser.add_argument(
        "--val_max_target_length",
        type=int,
        default=None,
        help="The maximum total sequence length for validation "
        "target text after tokenization.Sequences longer than this will be truncated, sequences shorter will be "
        "padded. Will default to `max_target_length`.This argument is also used to override the ``max_length`` "
        "param of ``model.generate``, which is used during ``evaluate`` and ``predict``.",
    )
    parser.add_argument(
        "--num_beams",
        type=int,
        default=None,
        help="Number of beams to use for evaluation. This argument will be "
        "passed to ``model.generate``, which is used during ``evaluate`` and ``predict``.",
    )
    parser.add_argument(
        "--model_name_or_path",
        type=str,
        help="Path to pretrained model or model identifier from huggingface.co/models.",
        required=False,
    )
    parser.add_argument(
        "--config_name",
        type=str,
        default=None,
        help="Pretrained config name or path if not the same as model_name",
    )
    parser.add_argument(
        "--tokenizer_name",
        type=str,
        default=None,
        help="Pretrained tokenizer name or path if not the same as model_name",
    )
    parser.add_argument(
        "--text_column",
        type=str,
        default=None,
        help="The name of the column in the datasets containing the full texts (for summarization).",
    )
    parser.add_argument(
        "--summary_column",
        type=str,
        default=None,
        help="The name of the column in the datasets containing the summaries (for summarization).",
    )
    parser.add_argument(
        "--use_slow_tokenizer",
        action="store_true",
        help="If passed, will use a slow tokenizer (not backed by the 🤗 Tokenizers library).",
    )
    parser.add_argument(
        "--per_device_train_batch_size",
        type=int,
        default=8,
        help="Batch size (per device) for the training dataloader.",
    )
    parser.add_argument(
        "--per_device_eval_batch_size",
        type=int,
        default=8,
        help="Batch size (per device) for the evaluation dataloader.",
    )
    parser.add_argument(
        "--learning_rate",
        type=float,
        default=5e-5,
        help="Initial learning rate (after the potential warmup period) to use.",
    )
    parser.add_argument("--weight_decay", type=float, default=0.0, help="Weight decay to use.")
    parser.add_argument("--num_train_epochs", type=int, default=3, help="Total number of training epochs to perform.")
    parser.add_argument(
        "--max_train_steps",
        type=int,
        default=None,
        help="Total number of training steps to perform. If provided, overrides num_train_epochs.",
    )
    parser.add_argument(
        "--gradient_accumulation_steps",
        type=int,
        default=1,
        help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument(
        "--lr_scheduler_type",
        type=SchedulerType,
        default="linear",
        help="The scheduler type to use.",
        choices=["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"],
    )
    parser.add_argument(
        "--num_warmup_steps", type=int, default=0, help="Number of steps for the warmup in the lr scheduler."
    )
    parser.add_argument("--output_dir", type=str, default=None, help="Where to store the final model.")
    parser.add_argument("--seed", type=int, default=None, help="A seed for reproducible training.")
    parser.add_argument(
        "--model_type",
        type=str,
        default=None,
        help="Model type to use if training from scratch.",
        choices=MODEL_TYPES,
    )
    parser.add_argument(
        "--overwrite_cache", type=bool, default=False, help="Overwrite the cached training and evaluation sets"
    )
    parser.add_argument(
        "--max_length", type=int, default=128, help="max length"
    )
    parser.add_argument(
        "--pad_to_max_length", type=bool, default=True, help="do pading"
    )
    parser.add_argument(
        "--ignore_pad_token_for_loss", type=bool, default=True, help="do pading"
    )
    parser.add_argument(
        "--logging_steps", type=int, default=500, help="do pading"
    )
    parser.add_argument(
        "--save_steps", type=int, default=5000, help="do pading"
    )
    parser.add_argument(
        "--save_every_checkpoint", action="store_true"
    )
    parser.add_argument(
        "--max_grad_norm", type=float, default=1.0, help="max_grad_norm"
    )
    parser.add_argument(
        "--no_kb", action="store_true"
    )
    parser.add_argument(
        "--exp_name",
        type=str,
        help="Description to the experiment",
        default='exp',
    )

    # Newly added params
    args, unknown = parser.parse_known_args()
    args.model_name_or_path = 'microsoft/GODEL-v1_1-base-seq2seq'
    args.train_file = 'perchat/perchat_train.jsonl'
    args.validation_file = 'perchat/perchat_valid.jsonl'
    args.dataset_config_name = 'perchat_dataset.py'
    # args.dataset_config_name = '../examples/dstc9/dstc9_dataset.py'
    # args.train_file = 'dstc9_train.jsonl'
    # args.validation_file = 'dstc9_val.jsonl
    args.per_device_train_batch_size = 16
    args.per_device_eval_batch_size = 16
    args.max_target_length = 128
    args.max_length = 512
    args.preprocessing_num_workers = 24
    args.num_beams = 5

    # Sanity checks
    if args.dataset_name is None and args.train_file is None and args.validation_file is None:
        raise ValueError("Need either a dataset name or a training/validation file.")
    else:
        if args.train_file is not None:
            extension = args.train_file.split(".")[-1]
            assert extension in ["csv", "json", "jsonl"], "`train_file` should be a csv or a json file."
        if args.validation_file is not None:
            extension = args.validation_file.split(".")[-1]
            assert extension in ["csv", "json", "jsonl"], "`validation_file` should be a csv or a json file."

    if args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)

    return args

In [34]:
# Decoding for DialoGLM
args = parse_args()

In [35]:
# Not necessary for generation
accelerator = Accelerator()
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)

INFO:__main__:Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Mixed precision type: no

INFO:__main__:Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Mixed precision type: no



In [36]:
if args.seed is not None:
    set_seed(args.seed)

In [37]:
if args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    # TODO: ?
    raw_datasets = load_dataset(args.dataset_name, args.dataset_config_name)
else:
  data_files = {}
  if args.train_file is not None:
      data_files["train"] = args.train_file
  if args.validation_file is not None:
      data_files["validation"] = args.validation_file
  extension = args.dataset_config_name
  raw_datasets = load_dataset(extension, data_files=data_files)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): storage.googleapis.com:443
DEBUG:urllib3.connectionpool:https://storage.googleapis.com:443 "HEAD /huggingface-nlp/cache/datasets/perchat_dataset/default-5ce1eb2bc68affa8/0.0.0/dataset_info.json HTTP/1.1" 404 0


Generating train split: 0 examples [00:00, ? examples/s]

DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/perchat_dataset/default-5ce1eb2bc68affa8/0.0.0/5eecee0de3cca5d81109673415673328813dd28b4d8835e4bb3528659c76425d.incomplete/perchat_dataset-train-00000-00000-of-NNNNN.arrow


Generating validation split: 0 examples [00:00, ? examples/s]

DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/perchat_dataset/default-5ce1eb2bc68affa8/0.0.0/5eecee0de3cca5d81109673415673328813dd28b4d8835e4bb3528659c76425d.incomplete/perchat_dataset-validation-00000-00000-of-NNNNN.arrow


Generating test split: 0 examples [00:00, ? examples/s]

DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/perchat_dataset/default-5ce1eb2bc68affa8/0.0.0/5eecee0de3cca5d81109673415673328813dd28b4d8835e4bb3528659c76425d.incomplete/perchat_dataset-test-00000-00000-of-NNNNN.arrow
DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/perchat_dataset/default-5ce1eb2bc68affa8/0.0.0/5eecee0de3cca5d81109673415673328813dd28b4d8835e4bb3528659c76425d.incomplete/dataset_info.json


Dataset perchat_dataset downloaded and prepared to /root/.cache/huggingface/datasets/perchat_dataset/default-5ce1eb2bc68affa8/0.0.0/5eecee0de3cca5d81109673415673328813dd28b4d8835e4bb3528659c76425d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [38]:
# Load pretrained model and tokenizer
if args.config_name:
    config = AutoConfig.from_pretrained(args.config_name)
elif args.model_name_or_path:
    config = AutoConfig.from_pretrained(args.model_name_or_path)
else:
    config = CONFIG_MAPPING[args.model_type]()
    logger.warning("You are instantiating a new config instance from scratch.")

if args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=not args.use_slow_tokenizer)
elif args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=not args.use_slow_tokenizer)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )

if args.model_name_or_path:
    model = AutoModelForSeq2SeqLM.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
    )
else:
    logger.info("Training new model from scratch")
    model = AutoModelForSeq2SeqLM.from_config(config)

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))
if model.config.decoder_start_token_id is None:
    raise ValueError("Make sure that `config.decoder_start_token_id` is correctly defined")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /microsoft/GODEL-v1_1-base-seq2seq/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /microsoft/GODEL-v1_1-base-seq2seq/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


In [39]:
prefix = args.source_prefix if args.source_prefix is not None else ""
padding = "max_length" if args.pad_to_max_length else False
max_target_length = args.max_target_length

def preprocess_function(examples):
    contextes = examples['Context']
    responses = examples['Response']
    kbs = examples['Knowledge']

    inputs = []
    for context, response, kb in zip(contextes, responses, kbs):
        if args.no_kb:
            inputs.append(prefix + ' ' + ' ' + context + ' => ')
        else:
            inputs.append(context + ' <|Knowledge|> ' + kb + ' => ')

    model_inputs = tokenizer(inputs, max_length=args.max_length, padding=padding, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(responses, max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length" and args.ignore_pad_token_for_loss:
        labels["labels"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["labels"]

    return model_inputs

In [40]:
# Processing dataset:
column_names = ['Context','Knowledge','Response']
lm_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=column_names,
    num_proc=args.preprocessing_num_workers,
    load_from_cache_file=False,
    desc=f"Processing dataset",
)

train_dataset = lm_datasets["train"]
eval_dataset = lm_datasets["validation"]
test_dataset = lm_datasets["test"]

Processing dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing dataset #5:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Processing dataset #21:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing dataset #22:   0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


Processing dataset #23:   0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "
DEBUG:fsspec.local:open 

In [41]:
eval_dataset[0]

{'input_ids': [33,
  25,
  3,
  9,
  207,
  568,
  42,
  3,
  9,
  1282,
  568,
  572,
  3,
  58,
  3,
  2,
  9175,
  439,
  7651,
  13553,
  9175,
  3155,
  3,
  23,
  317,
  48,
  131,
  12192,
  21,
  1402,
  125,
  773,
  13,
  568,
  25,
  33,
  3,
  5,
  754,
  442,
  72,
  32100,
  125,
  103,
  25,
  1243,
  57,
  96,
  10553,
  841,
  45,
  271,
  26676,
  96,
  3,
  58,
  841,
  429,
  36,
  26676,
  45,
  3182,
  396,
  231,
  68,
  3,
  99,
  79,
  33,
  1095,
  28,
  34,
  3,
  6,
  11,
  59,
  11233,
  120,
  3,
  13403,
  119,
  151,
  3,
  6,
  3,
  23,
  278,
  3,
  31,
  3,
  17,
  217,
  34,
  38,
  82,
  682,
  3,
  5,
  3,
  99,
  79,
  1663,
  12,
  240,
  8,
  1020,
  13,
  578,
  3,
  9,
  10951,
  27617,
  3,
  6,
  24,
  3,
  31,
  3,
  7,
  1399,
  57,
  140,
  3,
  5,
  32100,
  317,
  81,
  572,
  3,
  23,
  241,
  34,
  3,
  5,
  1664,
  3,
  23,
  54,
  3,
  31,
  3,
  17,
  253,
  136,
  1053,
  24,
  3,
  23,
  700,
  241,
  34,
  3,
  6,
  258,
  3,
  

In [42]:
raw_datasets['validation'][0]

{'Context': 'are you a good person or a bad person why ?',
 'Response': 'i am true neutral',
 'Knowledge': 'i think this just speaks for itself what kind of person you are . please post more EOS what do you mean by " excuse someone from being overweight " ? someone might be overweight from eating too much but if they are happy with it , and not adversely affecting other people , i don \' t see it as my problem . if they wish to take the risk of having a shorter lifespan , that \' s fine by me . EOS think about why i want it . sometimes i can \' t find any reason that i actually want it , then i let it slide . obviously with things like filing my taxes , the thought of a great big legal mess does the trick . EOS much of asia is like a magic fridge that when you open it contains whatever delicious food you want to eat , because you can almost always get something tasty at any time of day of night . and if you agent careful you might also get indigestion EOS can \' t speak for other peopl

In [43]:
# Log a few random samples from the training set:
for index in random.sample(range(len(train_dataset)), 1):
    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

label_pad_token_id = -100 if args.ignore_pad_token_for_loss else tokenizer.pad_token_id
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8 if accelerator.use_fp16 else None,
)

INFO:__main__:Sample 17940 of the training set: {'input_ids': [25, 3, 31, 3, 162, 118, 787, 8, 2491, 13, 492, 3, 9, 1355, 3047, 3, 9, 26, 21, 3, 9, 556, 13, 39, 4622, 3, 5, 25, 3, 31, 3, 162, 118, 787, 46, 11875, 1487, 11, 3, 25281, 7556, 42, 17086, 25, 241, 21, 48, 3, 9, 26, 3, 5, 125, 103, 25, 103, 3, 58, 3, 2, 9175, 439, 7651, 13553, 9175, 3155, 3115, 59, 44, 166, 593, 68, 3, 99, 25, 33, 1556, 305, 15, 258, 6729, 14231, 2134, 12, 59, 36, 38, 4421, 13, 3, 9, 2870, 788, 12, 8, 1041, 2717, 3, 5, 3, 99, 8, 7930, 737, 3, 31, 3, 17, 43, 3, 9, 418, 13, 3, 107, 102, 11, 737, 3, 31, 3, 17, 1560, 182, 614, 258, 2087, 8, 1921, 47, 8, 3, 9, 75, 3, 5, 3, 23, 317, 3, 9, 9858, 13, 3, 9206, 133, 36, 72, 2016, 68, 3, 99, 25, 3413, 17025, 376, 258, 3, 23, 3382, 34, 1279, 91, 3, 5, 82, 163, 822, 19, 410, 8, 3, 29, 102, 75, 28, 25, 143, 136, 1750, 3, 58, 3, 99, 78, 62, 3, 31, 3, 60, 79, 8, 1053, 25, 751, 3, 58, 8858, 8, 3, 26, 51, 19, 29, 3, 31, 3, 17, 1556, 376, 38, 3, 9, 3, 26, 51, 3, 102, 75, 3, 5, 

In [44]:
def postprocess_text(preds, labels):

    preds = [normalize_answer(pred.strip().replace('Agent :','')) for pred in preds]
    labels = [normalize_answer(label.strip().replace('Agent :','')) for label in labels]

    return preds, labels

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=data_collator, batch_size=args.per_device_train_batch_size
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size)
test_dataloader = DataLoader(test_dataset, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size)

In [45]:
# Optimizer
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader, test_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, test_dataloader
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [46]:
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
else:
    args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps,
    num_training_steps=args.max_train_steps,
)

if args.val_max_target_length is None:
    args.val_max_target_length = args.max_target_length

gen_kwargs = {
    "max_length": args.val_max_target_length if args is not None else config.max_length,
    "num_beams": args.num_beams,
}

In [47]:
# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader, test_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, test_dataloader
)

In [48]:
def evaluate_data(dataloader, eval_name='valid'):

    metric = load_metric("./utils/rouge_metric.py")
    metric_bleu = load_metric("./utils/bleu_metric.py")

    decoded_preds_extended = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                **gen_kwargs,
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch["labels"]
            if not args.pad_to_max_length:
                # If we did not pad to max length, we need to pad the labels too
                labels = accelerator.pad_across_processes(batch["labels"], dim=1, pad_index=tokenizer.pad_token_id)

            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()

            if args.ignore_pad_token_for_loss:
                # Replace -100 in the labels as we can't decode them.
                labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
            metric.add_batch(predictions=decoded_preds, references=decoded_labels)
            _decoded_preds = [i.split() for i in decoded_preds]
            _decoded_labels = [[i.split()] for i in decoded_labels]
            decoded_preds_extended.extend(_decoded_preds)
            metric_bleu.add_batch(predictions=_decoded_preds, references=_decoded_labels)


    result = metric.compute(use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result = {k: round(v, 4) for k, v in result.items()}
    logger.info(result)

    result_bleu = metric_bleu.compute()
    logger.info(result_bleu)

    if args.output_dir is not None:
        accelerator.wait_for_everyone()
        if accelerator.is_local_main_process:
            if not os.path.exists(args.output_dir):
                os.makedirs(args.output_dir)
            output_dir_file_name = os.path.join(args.output_dir, f'{eval_name}-results.json')
            print(output_dir_file_name)
            json.dump(decoded_preds_extended, open(output_dir_file_name,'w'), indent=2)
            logger.info("Saving model outputs to %s", output_dir_file_name)

In [ ]:
# Train!
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {args.num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {args.max_train_steps}")
# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
global_steps = 0
tr_loss, logging_loss = 0.0, 0.0
for epoch in range(args.num_train_epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
        global_steps += 1
        outputs = model(**batch)
        loss = outputs.loss
        loss = loss / args.gradient_accumulation_steps
        tr_loss += loss.item()
        accelerator.backward(loss)

        if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1

        if completed_steps >= args.max_train_steps:
            break

        if step % args.logging_steps == 0:
            logger.info(f"  EVALERR:  {(tr_loss - logging_loss)/float(args.logging_steps)}")
            if accelerator.is_local_main_process and USE_WANDB:
                wandb.log({'loss': tr_loss - logging_loss})
            logging_loss = tr_loss
            progress_bar.update(args.logging_steps)

        if args.output_dir is not None and global_steps % args.save_steps == 0 and global_steps > 0:
            accelerator.wait_for_everyone()
            if accelerator.is_local_main_process:
                checkpoint_prefix = 'checkpoint'
                output_dir = os.path.join(args.output_dir, '{}-{}'.format(checkpoint_prefix, global_steps))
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                unwrapped_model = accelerator.unwrap_model(model)
                unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

                tokenizer.save_pretrained(output_dir)
                torch.save(args, os.path.join(output_dir, 'training_args.bin'))
                logger.info("Saving model checkpoint to %s", output_dir)

    model.eval()

    gen_kwargs = {
        "max_length": config.max_length,
        "num_beams": args.num_beams,
    }

    def evaluate_data(dataloader, eval_name='valid'):

        metric_rouge = load_metric("./utils/rouge_metric.py")
        metric_bleu = load_metric("./utils/bleu_metric.py")

        decoded_preds_all = []
        for step, batch in enumerate(dataloader):
            with torch.no_grad():
                generated_tokens = accelerator.unwrap_model(model).generate(
                    batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    **gen_kwargs,
                )

                generated_tokens = accelerator.pad_across_processes(
                    generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
                )
                labels = batch["labels"]
                if not args.pad_to_max_length:
                    # If we did not pad to max length, we need to pad the labels too
                    labels = accelerator.pad_across_processes(batch["labels"], dim=1, pad_index=tokenizer.pad_token_id)

                generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
                labels = accelerator.gather(labels).cpu().numpy()

                if args.ignore_pad_token_for_loss:
                    # Replace -100 in the labels as we can't decode them.
                    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
                if isinstance(generated_tokens, tuple):
                    generated_tokens = generated_tokens[0]
                decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
                decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

                decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
                metric_rouge.add_batch(predictions=decoded_preds, references=decoded_labels)
                _decoded_preds = [i.split() for i in decoded_preds]
                _decoded_labels = [[i.split()] for i in decoded_labels]
                decoded_preds_all.extend(_decoded_preds)
                metric_bleu.add_batch(predictions=_decoded_preds, references=_decoded_labels)

        result = metric_rouge.compute(use_stemmer=True)
        result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
        result = {k: round(v, 4) for k, v in result.items()}
        logger.info(result)

        result_bleu = metric_bleu.compute()
        logger.info(result_bleu)

        accelerator.wait_for_everyone()
        if accelerator.is_local_main_process and USE_WANDB:
            wandb.log({f'{eval_name}_bleu': result_bleu['bleu']})
            wandb.log({f'{eval_name}_rouge': result['rougeL']})

        if args.output_dir is not None:
            accelerator.wait_for_everyone()
            if accelerator.is_local_main_process:
                if not os.path.exists(args.output_dir):
                    os.makedirs(args.output_dir)
                output_dir_file_name = os.path.join(args.output_dir, f'{eval_name}-step-{completed_steps}')
                json.dump(decoded_preds_all, open(output_dir_file_name,'w'), indent=2)
                logger.info("Saving model outputs to %s", output_dir_file_name)

    evaluate_data(eval_dataloader,'valid')
    evaluate_data(test_dataloader,'test')

    if args.output_dir is not None and args.save_every_checkpoint:
        accelerator.wait_for_everyone()

        if accelerator.is_local_main_process:
            checkpoint_prefix = 'checkpoint'
            output_dir = os.path.join(args.output_dir, '{}-epoch-{}'.format(checkpoint_prefix, completed_steps))
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

            tokenizer.save_pretrained(output_dir)
            torch.save(args, os.path.join(output_dir, 'training_args.bin'))
            logger.info("Saving model checkpoint to %s", output_dir)

INFO:__main__:***** Running training *****
INFO:__main__:  Num examples = 18907
INFO:__main__:  Num Epochs = 3
INFO:__main__:  Instantaneous batch size per device = 8
INFO:__main__:  Total train batch size (w. parallel, distributed & accumulation) = 8
INFO:__main__:  Gradient Accumulation steps = 1
INFO:__main__:  Total optimization steps = 7092


  0%|          | 0/7092 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
INFO:__main__:  EVALERR:  0.0072582802772521975
INFO:__main__:  EVALERR:  3.0952024595737457
INFO:__main__:  EVALERR:  3.0906814930438995
INFO:__main__:  EVALERR:  3.047716758966446
INFO:__main__:  EVALERR:  3.0215079259872435
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): github.com:443
DEBUG:urllib3.connectionpool:https://github.com:443 "HEAD /tensorflow/nmt/raw/master/nmt/scripts/bleu.py HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent

DEBUG:fsspec.local:open file: /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow
DEBUG:fsspec.local:open file: /root/.cache/huggingface/metrics/bleu/default/default_experiment-1-0.arrow
INFO:absl:Using default tokenizer.
INFO:__main__:{'rouge1': 9.3113, 'rouge2': 3.0188, 'rougeL': 8.8116, 'rougeLsum': 8.8126}
INFO:__main__:{'bleu': 0.03781557055280153, 'precisions': [0.12107597816196543, 0.044374920188992466, 0.036244541484716154, 0.03561759729272419], 'brevity_penalty': 0.7368769252992307, 'length_ratio': 0.7660872217139372, 'translation_length': 17584, 'reference_length': 22953}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): github.com:443
DEBUG:urllib3.connectionpool:https://github.com:443 "HEAD /tensorflow/nmt/raw/master/nmt/scripts/bleu.py HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /tensorflow/nmt/mast